In [9]:
import javalang
import time
import json
import os
from tqdm.notebook import tqdm

In [10]:
data_template = {
    "font-normal" : "JetBrainsMono-Regular.ttf",
    "font-bold" : "JetBrainsMono-Bold.ttf",
    "font-italic" : "JetBrainsMono-BoldItalic.ttf",
    "font-bold-italic" : "JetBrainsMono-Italic.ttf",
    "font-size" : 14,
    "width-margin" : 5,
    "height-margin" : 5,
    "spacing" : 1,
    "background-color" : [255, 255, 255],
}

In [15]:
source_folder = os.getcwd() + "./data/Snippets"
generator_folder = os.getcwd() + "./data/Generators"
files = [file for file in os.listdir(source_folder) if file.endswith(".java")]

for file in tqdm(files):
    f = open(source_folder + "/" + file)
    result = ""
    tokens = javalang.tokenizer.tokenize(f.read())
    base_line = None
    base_column = None
    flag = True
    for token in tokens:
        token_name = token.__class__.__name__
        token_value = token.value
        token_span = token.position
        if base_line is None:
            base_line = token_span[0]
        while token_span[0] > base_line:
            base_line += 1
            result += "\n"
            flag = True
        if flag:
            result += " " * token_span[1]
            base_column = token_span[1]
            flag = False
        while token_span[1] > base_column:
            result += " "
            base_column += 1
        result += "[aoi={}]{}[/aoi={}]".format(token_name, token_value, token_name)
        #result += "{}".format(token_value)
        base_column += len(token_value)
    data = data_template.copy()
    result = result.split("\n")
    data["source-code"] = result
    file_data = json.dumps(data, indent=4, separators=(',', ': '))
    file_name, ext = os.path.splitext(file)
    # write to file
    with open(generator_folder + "/" + file_name + "_ast.json", "w") as f:
        f.write(file_data)

  0%|          | 0/32 [00:00<?, ?it/s]